### Create readme.txt file

In [10]:
import os
import sys
import numpy as np
import torch

sys.path.append(os.path.abspath('../helper'))
import config
import utils
import loader
import plotfig

In [16]:
source_models_dir = "../model/static/CNN/BS16/3500_3516/ver5_"
target_data_dir   = "../../CDL Customization/Data/ver19_"

# create readme.txt file
content = f"""Generated by file 'transfer/test_transfer_(...).ipynb'.
Source models were loaded in {source_models_dir},
Target training data are loaded in {target_data_dir}
"""

idx_save_path = loader.find_incremental_filename('transferd_model/static/CNN', 'ver', '_', '')
transferd_save_path = f"transferd_model/static/CNN/ver{idx_save_path}_"

if not os.path.exists(os.path.dirname(transferd_save_path + '/readme.txt')):
    os.makedirs(os.path.dirname(transferd_save_path + '/readme.txt'))

# Write content to readme.txt
with open(transferd_save_path + '/readme.txt', "w") as file:
    file.write(content)

In [22]:
# Configuration
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 32 #64  # Batch size
NUM_EPOCHS = 20 # 20
learning_rate = 0.00001 # 1e-5
SNR = [0] # np.arange(0, 31, 5) # 0:5:30 dB

for snr in SNR:
    print(f" SNR: {snr}/{SNR[-1]}")
    for model_name in ['LS_CNN', 'LS_LI_CNN']:
        print(f'{model_name} model')
        # source model
        model_source = utils.CNN_Est().to(device)
        optimizer = torch.optim.Adam(model_source.parameters(), lr=learning_rate)
        
        checkpoint = torch.load(os.path.join(source_models_dir, f'{snr}dB', f'CNN_1_{model_name}_model.pth'))
        model_source.load_state_dict(checkpoint['model_state_dict'])
        print(model_source)

 SNR: 0/0
LS_CNN model
CNN_Est(
  (normalization): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(1, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
  (activate): ReLU()
  (conv2): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv4): Conv2d(64, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv5): Conv2d(32, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
)
LS_LI_CNN model
CNN_Est(
  (normalization): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(1, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
  (activate): ReLU()
  (conv2): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv4): Conv2d(64, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv5): Co

In [29]:
for name, param in model_source.named_parameters():
    print(f"Layer: {name}, Requires Grad: {param.requires_grad}")
    
num_params = sum(p.numel() for p in model_source.parameters() if p.requires_grad)
print(f"\nTotal number of trainable parameters: {num_params}")

Layer: normalization.weight, Requires Grad: True
Layer: normalization.bias, Requires Grad: True
Layer: conv1.weight, Requires Grad: False
Layer: conv1.bias, Requires Grad: False
Layer: conv2.weight, Requires Grad: False
Layer: conv2.bias, Requires Grad: False
Layer: conv3.weight, Requires Grad: False
Layer: conv3.bias, Requires Grad: False
Layer: conv4.weight, Requires Grad: False
Layer: conv4.bias, Requires Grad: False
Layer: conv5.weight, Requires Grad: True
Layer: conv5.bias, Requires Grad: True

Total number of trainable parameters: 803


In [30]:
for name, param in model_source.named_parameters():
    if any(f"conv{i}" in name for i in range(1, 5)):
        param.requires_grad = False

for name, param in model_source.named_parameters():
    print(f"Layer: {name}, Requires Grad: {param.requires_grad}")

num_params = sum(p.numel() for p in model_source.parameters() if p.requires_grad)
print(f"\nTotal number of trainable parameters: {num_params}")

Layer: normalization.weight, Requires Grad: True
Layer: normalization.bias, Requires Grad: True
Layer: conv1.weight, Requires Grad: False
Layer: conv1.bias, Requires Grad: False
Layer: conv2.weight, Requires Grad: False
Layer: conv2.bias, Requires Grad: False
Layer: conv3.weight, Requires Grad: False
Layer: conv3.bias, Requires Grad: False
Layer: conv4.weight, Requires Grad: False
Layer: conv4.bias, Requires Grad: False
Layer: conv5.weight, Requires Grad: True
Layer: conv5.bias, Requires Grad: True

Total number of trainable parameters: 803
